## 1. Check if the model is differentiable

In [8]:
import torch
from torch.nn.parameter import Parameter

class QubeDynamics(torch.nn.Module):
    """Solve equation M qdd + C(q, qd) = tau for qdd."""

    def __init__(self):
        super().__init__()
        # Gravity
        self.g = Parameter(data=torch.Tensor([9.81]), requires_grad=True)

        # Motor
        self.Rm = Parameter(data=torch.Tensor([8.4]), requires_grad=True)

        # back-emf constant (V-s/rad)
        self.km = Parameter(data=torch.Tensor([0.042]), requires_grad=True)

        # Rotary arm
        self.Mr = Parameter(data=torch.Tensor([0.095]), requires_grad=True)
        self.Lr = Parameter(data=torch.Tensor([0.085]), requires_grad=True)
        self.Dr = Parameter(data=torch.Tensor([5e-6]), requires_grad=True)

        # Pendulum link
        self.Mp = Parameter(data=torch.Tensor([0.024]), requires_grad=True)
        self.Lp = Parameter(data=torch.Tensor([0.129]), requires_grad=True)
        self.Dp = Parameter(data=torch.Tensor([1e-6]), requires_grad=True)

        # Init constants
        self._init_const()

    def _init_const(self):
        # Moments of inertia
        Jr = self.Mr * self.Lr ** 2 / 12  # inertia about COM (kg-m^2)
        Jp = self.Mp * self.Lp ** 2 / 12  # inertia about COM (kg-m^2)

        # Constants for equations of motion
        self._c = torch.zeros(5)
        self._c[0] = Jr + self.Mp * self.Lr ** 2
        self._c[1] = 0.25 * self.Mp * self.Lp ** 2
        self._c[2] = 0.5 * self.Mp * self.Lp * self.Lr
        self._c[3] = Jp + self._c[1]
        self._c[4] = 0.5 * self.Mp * self.Lp * self.g


    def forward(self, s, u):
        th, al, thd, ald = s
        voltage = u[0]

        # Define mass matrix M = [[a, b], [b, c]]
        a = self._c[0] + self._c[1] * torch.sin(al) ** 2
        b = self._c[2] * torch.cos(al)
        c = self._c[3]
        d = a * c - b * b

        # Calculate vector [x, y] = tau - C(q, qd)
        trq = self.km * (voltage - self.km * thd) / self.Rm
        c0 = self._c[1] * torch.sin(2 * al) * thd * ald \
            - self._c[2] * torch.sin(al) * ald * ald
        c1 = -0.5 * self._c[1] * torch.sin(2 * al) * thd * thd \
            + self._c[4] * torch.sin(al)
        x = trq - self.Dr * thd - c0
        y = -self.Dp * ald - c1

        # Compute M^{-1} @ [x, y]
        thdd = (c * x - b * y) / d
        aldd = (a * y - b * x) / d

        return torch.Tensor([thdd, aldd])

In [20]:
model = QubeDynamics()
initial_state = torch.Tensor([0.0, 0.0, 0.0, 0.0])

# run model
thdd, aldd = model(initial_state, torch.Tensor([0.0]))

# update state
dt = 0.02
next_state = torch.clone(initial_state)
next_state[3] += (dt * aldd)[0]
next_state[2] += (dt * thdd)[0]
next_state[1] += dt * next_state[3]
next_state[0] += dt * next_state[2]

real_next_state = torch.Tensor([1.0, 1.0, 1.0, 1.0]) # dummy values

loss = torch.nn.functional.mse_loss(next_state, real_next_state)
print(loss)
loss.backward()

tensor(1., grad_fn=<MseLossBackward0>)
